In [165]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np

In [155]:
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2022, 12, 31)
ticker = "AAPL"

In [163]:
train_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [158]:
print(type(train_data))

<class 'pandas.core.frame.DataFrame'>


In [164]:
train_data["daily_return"] = train_data["Adj Close"].pct_change()
train_data["state"] = np.select(
    [train_data["daily_return"] > 0.005, train_data["daily_return"] < -0.005],
    ["up", "down"],
    default="neutral"
)
train_data.head(15)

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2000-01-03,148.25000,148.25000,143.875000,145.43750,94.262505,8164300,NaN,neutral
2000-01-04,143.53125,144.06250,139.640625,139.75000,90.576309,8089800,-0.039106,down
2000-01-05,139.93750,141.53125,137.250000,140.00000,90.738319,12177900,0.001789,neutral
2000-01-06,139.62500,141.50000,137.750000,137.75000,89.280075,6227200,-0.016071,down
2000-01-07,140.31250,145.75000,140.062500,145.75000,94.465134,8066500,0.058076,up
2000-01-10,146.25000,146.90625,145.031250,146.25000,94.789169,5741700,0.003430,neutral
2000-01-11,145.81250,146.09375,143.500000,144.50000,93.654938,7503700,-0.011966,down
2000-01-12,144.59375,144.59375,142.875000,143.06250,92.723267,6907700,-0.009948,down
2000-01-13,144.46875,145.75000,143.281250,145.00000,93.979050,5158300,0.013543,up


In [166]:
up_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))

down_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))

neutral_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))

In [167]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_neutral, up_to_down],
    "neutral": [neutral_to_up, neutral_to_neutral, neutral_to_down],
    "down": [down_to_up, down_to_neutral, down_to_down]
}, index=["up", "neutral", "down"])
transition_matrix.head()
print(transition_matrix)

               up   neutral      down
up       0.264988  0.298927  0.295198
neutral  0.416067  0.524232  0.417373
down     0.318945  0.176471  0.287429


In [ ]:
data.head()

Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2020-01-02  74.059998  75.150002  73.797501  75.087502  73.152657  135480400   
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.441467  146322800   
2020-01-06  73.447502  74.989998  73.187500  74.949997  73.018677  118387200   
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.675278  108872000   
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.844353  132079200   

            daily_return state  
Date                            
2020-01-02           NaN  down  
2020-01-03     -0.009722  down  
2020-01-06      0.007968    up  
2020-01-07     -0.004703  down  
2020-01-08      0.016086    up

In [40]:
# Test Predictions
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 6, 1)
ticker = "AAPL"
test_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [42]:
portfolio_size = 1000

for i in range(1, len(train_data)):
    # If it goes down
    if (test_data[i-1]["Adj Close"] > test_data[i]["Adj Close"])):
        shares = portfolio_size / test_data[i]["Adj Close"]
        portfolio = shares * test_data[i+1]["Adj Close"]

print(portfolio_size)

KeyError: 0

In [170]:
from backtesting import Backtest, Strategy

In [172]:
from backtesting.test import GOOG

In [187]:
class MarkovStrategy(Strategy):
    def init(self):
        """init"""
        price = self.data.Close

    def next(self):
        self.position.close()
        self.buy()

In [188]:
bt = Backtest(GOOG, MarkovStrategy, cash=10000, commission=0)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    99.90689
Equity Final [$]                      72589.6
Equity Peak [$]                       72649.0
Return [%]                            625.896
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   26.181719
Volatility (Ann.) [%]               44.490485
Sharpe Ratio                         0.588479
Sortino Ratio                        1.169879
Calmar Ratio                         0.401173
Max. Drawdown [%]                  -65.262989
Avg. Drawdown [%]                   -5.463657
Max. Drawdown Duration     1784 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                                 2146
Win Rate [%]                        52.096925
Best Trade [%]                      18.102493
Worst Trade [%]                    -11.563496
Avg. Trade [%]                    

In [186]:
bt.plot()

/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2563', ...)